In [1]:
#!conda update -n base -c defaults conda

In [2]:
#!conda install -yc conda-forge ipython-sql

# MTV is dead:
## SQL for  business questions in multimedia app

![banner](https://i0.wp.com/liveforlivemusic.com/wp-content/uploads/2016/02/musicbrain.jpg?resize=610%2C390&ssl=1)

## Abstract

Mobile technology is changing we relate to other humans and objects around us. On media consumption, this is also the case, where we once owned media now most content is subcribtion-available, and that implies gigantic databases. In this project, we deploy SQLite to extract business insights from a fictitious music app, chinook. We query:

* 1. Most sold genre in USA;
* 2. How much each sales agent sold;
* 3. Calculate, for each country: number of customers, total value of sales, avg value per customer,avg order value;
* 4. What % of purchases are individual tracks vs whole albums.
* 5. Which artist is used the most in playlists?
* 6. How many tracks have been purchased vs not purchased?
* 7. Is the range of tracks in the store reflective of their sales popularity?
* 8. Do protected vs non-protected media types have an effect on popularity?

The results were as follows:

* 1. Rock (53%);
* 2. Around 1500 dollars, varies due to in-house experience;
* 3. Czechs are the most willing to spend, 50% more than following Portuguese and Indians.
* 4. Nearly 20% of purchases are albums.
* 5. Iron Maiden, U2, Metallica, Led Zeppelin and Deep Purple are the most popular artists in playlists, ranging from 2.6%-5.92% in participation.
* 6. 1679, or 48% of the available tracks have never been purchased;
* 7. There are genres under represented by 50% (Easy Listening, R&B/Soul, Alternative), over represented (Drama - 86x, TV Shows - 64x, Soundtracks - 11x), and even never sold: 'World', 'SciFI & Fantasy', 'Comedy', 'Bossa Nova', 'Science Fiction', 'Rock and Roll', 'Opera'
* 8. Protecting media types have no effect on popularity.

## Table of Contents

* 1. [**Introduction**](#section1)
* 2. [**Goals and limitations**](#section2)
* 3. [**Methodology**](#section3)
* 4. [**Metadata**](#section4)
* 5. [**Data Exploration and Analysis**](#section5)
*  5.1 [**Task 1: Selecting albums to purchase**](#section7)
*  5.2 [**Analyzing Employee Sales performance**](#section8)
*  5.3 [**Analyzing sales by country**](#section9)
*  5.4 [**Albums vs Individual tracks**](#section10)
*  5.5 [**Which artist is used the most in playlists?**](#section11)
*  5.6 [**How many tracks have been purchased vs not purchased?**](#section12)
*  5.7 [**Is the range of tracks in the store reflective of their sales popularity (by genre)?**](#section13)
*  5.8 [**How do protected files compare to the rest in sales performance?**](#section14)
* 6. [**Conclusion**](#section6)

## 1. Introduction<a name="section1"></a>

The advance of mobile technology has shifted many paradigms in the way we interact with the world around us. Distances have been shortened, information is easily shared and available and thus we have been much more absorbed in them. 

The landscape of entertainment consumption in the 21st century, for instance, is marked by the replacement of owning individual titles, such as DVDs and tapes, to subscription-based services. This expands individual access to content in exchange of heavy databases.

For this project, we explore a fictional music app database, [chinook](https://github.com/lerocha/chinook-database/tree/master/ChinookDatabase/DataSources), in order to address some business questions. It is fairly common in business to store data in multiple sheets, and so is the case here. Being able to combine such tables in order to summarize statistics is vital to the role of a Data Analyst.

## 2. Goals and limitations<a name="section2"></a>

We will look into the chinook database to answer business questions:

* 1. Which genres sell the best in the USA?
* 2. How much was assigned to each sales support agent?
* 3. Number of customers, total value of sales, avg value per customer,avg order value foreach country?
* 4. What % of purchases are individual tracks vs whole albums?
* 5. Which artist is used the most in playlists?
* 6. How many tracks have been purchased vs not purchased?
* 7. Is the range of tracks in the store reflective of their sales popularity?
* 8. Do protected vs non-protected media types have an effect on popularity?

Limitations:
* Lack of metadata;
* Unfamiliarity with SQL;
* No graphic interface deployed with SQL to better illustrate outcomes;
* Apparently, no wrong/missing values, maybe unrealistic in the real world.

## 3. Methodology<a name="section3"></a>

For this project, we will deploy SQLite in a db file to join and compile different tables into meaningful insights to be drawn.

## 4. Metadata<a name="section4"></a>

The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

Why the name Chinook?
The name of this sample database was based on the Northwind database. Chinooks are winds in the interior West of North America, where the Canadian Prairies and Great Plains meet various mountain ranges. Chinooks are most prevalent over southern Alberta in Canada. Chinook is a good name choice for a database that intents to be an alternative to Northwind.

Sample Data
Media related data was created using real data from an iTunes Library. Customer and employee information was manually created using fictitious names, addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.). Sales information is auto generated using random data for a four year period.

The database can be represented as the scheme bellow.
![banner](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

## 5. Data Exploration and Analysis<a name="section5"></a>

In [3]:
#%%capture
%load_ext sql
# %sql sqlite:///factbook.db
%sql sqlite:///chinook.db

'Connected: @chinook.db'

In [4]:
%%sql
SELECT 
    name,
    type
FROM sqlite_master
WHERE type IN ('table', 'view')

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### 5.1 Task 1: Selecting albums to purchase<a name="section7">
Write a query to find out which genres sell the best in the USA

In [5]:
%%sql
WITH 
    usa AS (
     SELECT * FROM customer
     WHERE country='USA'
    ),

    usa_bills AS ( 
     SELECT
         usa.customer_id,
         usa.company,
         usa.state,
         usa.country,
         t.track_id,
         i.invoice_id,
         il.unit_price,
         g.genre_id,
         g.name genre
     FROM usa
     LEFT JOIN invoice i ON i.customer_id = usa.customer_id 
     LEFT JOIN invoice_line il ON il.invoice_id=i.invoice_id    
     LEFT JOIN track t ON t.track_id=il.track_id
     LEFT JOIN genre g ON g.genre_id=t.genre_id)

SELECT
    ub.genre genre,
    COUNT(ub.genre_id) tracks_sold,
    ROUND(CAST(COUNT(ub.genre_id)AS FLOAT)*100/(SELECT COUNT(*) FROM usa_bills), 2) '%_sold_tracks',
    ROUND(SUM(ub.unit_price), 2) 'total_sales US$',
    ROUND(CAST(SUM(ub.unit_price) AS FLOAT)*100/(SELECT SUM(unit_price) FROM usa_bills), 2) '% of revenue'
FROM usa_bills ub
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,%_sold_tracks,total_sales US$,% of revenue
Rock,561,53.38,555.39,53.38
Alternative & Punk,130,12.37,128.7,12.37
Metal,124,11.8,122.76,11.8
R&B/Soul,53,5.04,52.47,5.04
Blues,36,3.43,35.64,3.43
Alternative,35,3.33,34.65,3.33
Pop,22,2.09,21.78,2.09
Latin,22,2.09,21.78,2.09
Hip Hop/Rap,20,1.9,19.8,1.9
Jazz,14,1.33,13.86,1.33


According to the data, the most sold and profitable music genre in the USA is by far Rock, accounting for 53% of the sales and revenue. Considering our 4 contenders, 

|Artist Name|	Genre|
| --- | --- |
|Regal|	Hip-Hop|
|Red Tone|	Punk|
|Meteor and the Girls|	Pop|
|Slim Jim Bites|	Blues|

we should promote, in order, Red Tone(Punk), Slim Jim Bites(Blues), Meteor and the Girls(Pop).

### 5.2. Analyzing Employee Sales performance<a name="section8">

Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis

In [6]:
%%sql 
WITH
    ssa AS (
    SELECT 
        e.employee_id,
        e.first_name||" "||e.last_name full_name,
        e.country,
        e.hire_date
    FROM employee e
    WHERE e.title IS 'Sales Support Agent'        
        ),
     money_per_customer AS (
     SELECT
         c.customer_id,
         c.state,
         c.country,
         c.company,
         i.invoice_id,
         i.total,
         c.support_rep_id
     FROM customer c
     LEFT JOIN invoice i ON i.customer_id=c.customer_id
         )
    
SELECT 
    mpc.support_rep_id,
    ROUND(SUM(mpc.total), 2) total_spent,
    ssa.hire_date    
FROM money_per_customer mpc
LEFT JOIN ssa ON ssa.employee_id = mpc.support_rep_id      
GROUP BY 1;

 * sqlite:///chinook.db
Done.


support_rep_id,total_spent,hire_date
3,1731.51,2017-04-01 00:00:00
4,1584.0,2017-05-03 00:00:00
5,1393.92,2017-10-17 00:00:00


We can attribute the difference in sales assigned to each agent to in-house experience.

### 5.3. Analyzing sales by country<a name="section9">

Calculate, for each country:
* total number of customers;
* total value of sales;
* avg value per customer;
* avg order value
(countries with 1 customer should be forced into 'Others' and stay at the bottom of list)

In [7]:
%%sql
WITH 
    rough AS (
        SELECT 
            c.country,
            c.customer_id  ,
            SUM(i.total) amount_ind,
            COUNT(i.invoice_id) orders_ind 
        FROM customer c
        LEFT JOIN invoice i ON i.customer_id=c.customer_id
        GROUP BY 2),
    exact AS (
        SELECT 
            r.country, 
            COUNT(r.customer_id) national_customers,
            ROUND(SUM(r.amount_ind), 2) national_revenue,
            ROUND(SUM(r.amount_ind)/COUNT(r.customer_id), 2) national_avg_customer,
            SUM(r.orders_ind) national_orders,
            ROUND(SUM(r.amount_ind)/SUM(r.orders_ind), 2) national_avg_order,
            CASE
                WHEN COUNT(r.customer_id) IS 1 THEN 1
                ELSE 0
            END AS one_customer
        FROM rough r
        GROUP BY 1)
    
SELECT 
    CASE
        WHEN ex.national_customers IS 1 THEN 'Others'
        ELSE ex.country
    END AS new_country,
    SUM(ex.national_customers) total_consumers,
    SUM(ex.national_revenue) total_sales,
    ROUND(SUM(ex.national_revenue)/SUM(ex.national_customers), 2) avg_customer,
    ROUND(SUM(ex.national_revenue)/SUM(ex.national_orders), 2) avg_order
FROM  exact ex
GROUP BY 1
ORDER BY ex.one_customer, 4 DESC;

 * sqlite:///chinook.db
Done.


new_country,total_consumers,total_sales,avg_customer,avg_order
Czech Republic,2,273.24,136.62,9.11
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Brazil,5,427.68,85.54,7.01
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
USA,13,1040.49,80.04,7.94
France,5,389.07,77.81,7.78
Canada,8,535.59,66.95,7.05
Others,15,1094.94,73.0,7.45


We can rank countries by their average bill per customer, showing us that Czech consumers are by far the most willing to spend (+50% over the 2nd ranked), followed by Portugal and India.

### 5.4 Albums vs Individual tracks<a name="section10">

    Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.
    
What % of purchases are individual tracks vs whole albums. Edge cases: 
* Albums that have only 1 or 2 tracks are likely to be purchased by customers as part of a collection of individual tracks;
* Customers may select every track from an album, and then a few.

In [8]:
%%sql 
WITH
    a_t AS(
    SELECT
        a.*,
        t.name track_name,
        t.track_id
    FROM album a
    INNER JOIN track t ON t.album_id = a.album_id
    ),
-- 3503
    a_t_sum AS (
    SELECT
        a.album_id,
        a.title,
        SUM(track_id) + album_id id_sum
    FROM a_t a
    GROUP BY 1 ORDER BY 1
    ),
 -- 347   
    il_i_t AS (
    SELECT*
    FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id=il.invoice_id
    INNER JOIN track t ON t.track_id = il.track_id
    ),
-- 4757
    full_a_t AS (
    SELECT
        a_t.*,
        a_t_sum.id_sum
    FROM a_t
    INNER JOIN a_t_sum ON a_t_sum.album_id = a_t.album_id),
-- 3503
    final_scope AS (
    SELECT
        ilit.invoice_id,
        fat.album_id,
        fat.artist_id,
        fat.id_sum,
        fat.track_id,
        fat.track_name,       
        ilit.quantity
    FROM full_a_t fat
    INNER JOIN il_i_t ilit ON ilit.track_id = fat.track_id),
--4757
    final_marked AS (
    SELECT 
        invoice_id,
        COUNT(DISTINCT(album_id)) 'albums/sale',
        COUNT(track_id) 'tracks/sale',
        id_sum,
        SUM(track_id)+album_id full_compare,
    CASE
        WHEN (SUM(track_id)+album_id) IS id_sum 
        THEN 1
        ELSE 0
    END AS full_album_sold
    FROM final_scope
    GROUP BY 1
        )
    
SELECT
    full_album_sold,
    COUNT(invoice_id) total_sales,
    ROUND((COUNT(invoice_id)*100.0)/(SELECT COUNT(id_sum) FROM final_marked), 2) '% of sales'
FROM final_marked
GROUP BY 1;

 * sqlite:///chinook.db
Done.


full_album_sold,total_sales,% of sales
0,500,81.43
1,114,18.57


Cutting on full albums in favor of only popular tracks risks 18.5% of purchases, so I'd recommend putting full albums aside for a more unanimously popular selection.

### 5.5 Which artist is used the most in playlists?<a name="section11">

In [9]:
%%sql
WITH
    pl_plt_t AS (
    SELECT 
        pl.playlist_id,
        pl.name playlist_name,
        pl_t.track_id
    FROM playlist pl
    INNER JOIN playlist_track pl_t ON pl_t.playlist_id=pl.playlist_id
    INNER JOIN track t ON t.track_id = pl_t.track_id
    ),
    t_al_ar AS (
    SELECT 
        t.track_id,
        t.name track_name,
        g.name genre,
        al.title album_title,
        ar.name artist_name,
        ar.artist_id,
        t.unit_price
    FROM track t 
    INNER JOIN album al ON t.album_id = al.album_id
    INNER JOIN artist ar ON ar.artist_id = al.artist_id 
    INNER JOIN genre g ON g.genre_id = t.genre_id
        ),
    combined AS (
    SELECT
        p.playlist_id,
        p.playlist_name,
        t.*
    FROM pl_plt_t p
    INNER JOIN t_al_ar t ON p.track_id=t.track_id)
    
SELECT 
    artist_name,
    COUNT(artist_id) songs_in_pls,
    ROUND(COUNT(artist_id)*100.0/(SELECT COUNT(artist_id) FROM combined),2) '%_in_pls'
FROM combined
GROUP BY artist_name
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


artist_name,songs_in_pls,%_in_pls
Iron Maiden,516,5.92
U2,333,3.82
Metallica,296,3.4
Led Zeppelin,252,2.89
Deep Purple,226,2.59
Lost,184,2.11
Pearl Jam,177,2.03
Faith No More,145,1.66
Eric Clapton,145,1.66
Lenny Kravitz,143,1.64


The top 5 most dominant artist in playlists in the Chinook database are Iron Maiden, U2, Metallica, Led Zeppelin and Deep Purple, ranging from 2.6% - 5.92% of all songs in users' playlists.

### 5.6 How many tracks have been purchased vs not purchased?<a name="section12">

In [10]:
%%sql

WITH
    grouped_il AS (
    SELECT 
        SUM(il.quantity) n_purchases,
        il.track_id
    FROM invoice_line il
    GROUP BY 2 ORDER BY 1 DESC
        ),
    summary AS (
    SELECT 
        t.track_id,
        t.name track_name,
        gi.n_purchases
                
    FROM track t
    LEFT JOIN grouped_il gi ON gi.track_id = t.track_id
    ORDER BY 3
        )
    
SELECT
    n_purchases,
    COUNT(track_id) total_tracks,
    ROUND(COUNT(track_id)*100.0/(SELECT COUNT(track_id) FROM summary), 2) perc_tracks
FROM summary s
GROUP BY 1 ORDER BY 1;

 * sqlite:///chinook.db
Done.


n_purchases,total_tracks,perc_tracks
None,1697,48.44
1,705,20.13
2,438,12.5
3,244,6.97
4,141,4.03
5,98,2.8
6,70,2.0
7,34,0.97
8,29,0.83
9,19,0.54


1679, or 48% of the available tracks have never been purchased. This represents tremendous unpaid investment in this library, and demands for change of management strategy.

### 5.7 Is the range of tracks in the store reflective of their sales popularity (by genre)?<a name="section13">    

In [11]:
%%sql
WITH
    geral AS (
    SELECT
        g.name genre,
        t.track_id,
        CASE
            WHEN SUM(il.quantity) IS NULL
            THEN 0
            ELSE SUM(il.quantity)
        END AS tracks_sold
    FROM track t
    INNER JOIN genre g ON g.genre_id=t.genre_id
    LEFT JOIN invoice_line il ON il.track_id = t.track_id
    GROUP BY 2
    ORDER BY 3
    )
    
SELECT 
    genre,
    COUNT(track_id) tracks_available,
    SUM(tracks_sold) tracks_sold,
    ROUND(COUNT(track_id)*100.0/(SELECT COUNT(track_id) FROM geral), 2) '%_available',    
    ROUND(SUM(tracks_sold)*100.0/(SELECT SUM(tracks_sold) FROM geral), 2) '%_sales',
    ROUND((COUNT(track_id)*100.0/(SELECT COUNT(track_id) FROM geral))/(SUM(tracks_sold)*100.0/(SELECT SUM(tracks_sold) FROM geral)), 2) 'rep_factor'
FROM geral
GROUP BY 1
ORDER BY 6 DESC, 2 DESC;

 * sqlite:///chinook.db
Done.


genre,tracks_available,tracks_sold,%_available,%_sales,rep_factor
Drama,64,1,1.83,0.02,86.91
TV Shows,93,2,2.65,0.04,63.15
Soundtrack,43,5,1.23,0.11,11.68
Heavy Metal,28,8,0.8,0.17,4.75
Latin,579,167,16.53,3.51,4.71
Reggae,58,35,1.66,0.74,2.25
Classical,74,47,2.11,0.99,2.14
Jazz,130,121,3.71,2.54,1.46
Hip Hop/Rap,35,33,1.0,0.69,1.44
Pop,48,63,1.37,1.32,1.03


We can see extremes when ranking genres by their representation in sales. Easy Listening, Alternative, R&B/Soul deserve to double their representation in the media menu given their sales. Drama, Soundtrack, Heavy Metal, Latin, and TV Shows range from 4.7 to 86 times more representation in content than in sales. There are even cases where full genres have never been purchased: 'World', 'SciFI & Fantasy', 'Comedy', 'Bossa Nova', 'Science Fiction', 'Rock and Roll', 'Opera', which account for 3.16% of the available content.

### 5.8 How do protected files compare to the rest in sales performance?<a name="section14">    

In [12]:
%%sql
WITH 
    my_scope AS (
    SELECT
        t.track_id,
        t.name track_name,
        mt.name media_type,
        CASE
            WHEN mt.name LIKE '%Protected%' THEN 1
            ELSE 0
        END AS protected,
        SUM(il.quantity) tracks_sold
    FROM track t
    INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
    LEFT JOIN invoice_line il ON il.track_id = t.track_id
    GROUP BY 1
    )
    
SELECT 
    protected,
    SUM(tracks_sold) tracks_sold,
    ROUND((SUM(tracks_sold)*100.0)/(SELECT SUM(tracks_sold) FROM my_scope),2) '%_tracks_sold', 
    COUNT(track_id) available_tracks,
    ROUND((COUNT(track_id)*100.0)/(SELECT COUNT(track_id) FROM my_scope), 2) '%_available_tracks'
FROM my_scope GROUP BY protected;

 * sqlite:///chinook.db
Done.


protected,tracks_sold,%_tracks_sold,available_tracks,%_available_tracks
0,4315,90.71,3052,87.13
1,442,9.29,451,12.87


It doesn't seem to matter whether the files are protected or not, given the popularity is very close in representation to the number of available tracks.

## 6. Conclusion<a name="section6"></a>

In this project, we managed to extract real insights from the chinook database using :

* The most popular music genre in the USA is by far Rock, accounting for 53% of the sales and revenue;
* We can attribute the difference in sales assigned to each agent to in-house experience;
* Czech consumers are by far the most willing to spend (+50% over the 2nd ranked), followed by Portugal and India;
* Approximately 18.5% of the sales are full albums, and there is potential to not display those in favour of more popular individual tracks.
* Iron Maiden, U2, Metallica, Led Zeppelin and Deep Purple are the most popular artists in playlists, ranging from 2.6%-5.92% in participation.
* 1679, or 48% of the available tracks have never been purchased;
* There are genres under represented by 50% (Easy Listening, R&B/Soul, Alternative), over represented (Drama - 86x, TV Shows - 64x, Soundtracks - 11x), and even never sold: 'World', 'SciFI & Fantasy', 'Comedy', 'Bossa Nova', 'Science Fiction', 'Rock and Roll', 'Opera'
* Protecting media types have no effect on popularity.